In [1]:
! pip install -q crewai crewai[tools] langchain-google-genai langchain-community

In [2]:
import os
from google.colab import userdata
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool
from langchain_google_genai import ChatGoogleGenerativeAI

In [46]:
SERPER_API_KEY = userdata.get('SERPER_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [47]:
search_tools = SerperDevTool()

In [48]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.5
)

In [49]:
researcher = Agent(
      role = "Senior Researcher",
      goal = "Uncover groundbreaking technologies in {topic}",
      backstory='''
      Driven by curiosity , you're at the forefront of innovation, eager to explore and share knowlegde that could change the world
      ''',
      verbose=True,
      allow_delegation=True,
      memory = True,
      tools = [search_tools],
      llm = llm
  )

LLM value is already an LLM object


In [50]:
writer = Agent(
      role = "Senior Writer",
      goal = "Narrate compelling tech stories about {topic}",
      backstory='''
      With a flair of simplifying complex topics, you craft engaging narratives that resonate with readers.
      ''',
      verbose=True,
      allow_delegation=False,
      memory = True,
      tools = [search_tools],
      llm = llm
  )

LLM value is already an LLM object


In [51]:
research_task = Task(
      description = "Research on the following {topic} and give conprehensive summary on it.",
      agent=researcher,
      tools = [search_tools],
      expected_output = "A detailed summary of the research finding, including key points and insights related to topic"
  )

In [52]:
writer_task = Task(
      description = "Create an article on {topic}",
      agent=writer,
      tools = [search_tools],
      expected_output = "A detailed summary of the research finding, including key points and insights related to {topic}",
      async_execution=False,
      output_file="new_blog_post.md"
  )

In [54]:
crew = Crew(
    agents = [researcher, writer],
    tasks = [research_task, writer_task]
)

result = crew.kickoff(inputs={"topic": "Large Language Model"})
print(result)

# Agent: Senior Researcher
## Task: Research on the following Large Language Model and give conprehensive summary on it.


# Agent: Senior Researcher
## Thought: ```tool_code
Thought:I need to identify the Large Language Model to research.  The prompt doesn't specify one.  I'll search the internet for prominent LLMs to get a list of options.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"list of prominent large language models\"}"
## Tool Output: 
{'searchParameters': {'q': 'list of prominent large language models', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'List of the Best 21 Large Language Models (LLMs) (September 2024)', 'link': 'https://explodingtopics.com/blog/list-of-llms', 'snippet': 'List of the Best 21 Large Language Models (LLMs) (September 2024) · 1. GPT-4o · 2. Claude 3.5 · 3. Grok-1 · 4. Mistral 7B · 5. PaLM 2 · 6.', 'position': 1}, {'title': '19 of the best large language models in 2024 - TechTarget', 'l